In [3]:
import seaborn as sns
import pandas as pd
import befaas.logentry as le
from befaas.fileutil import log_entry_iterator


sns.set()

filepath = "../dumps/"
#logdumps = ["dump_webservice_google_23.json","dump_webservice_aws_23.json","dump_webservice_azure_23.json"]
logdumps = ["dump_webservice_aws_small.json"]
oufile = "../img/functionRuntimesEcomm.pkl"
print("Everything is set up.")


Everything is set up.


In [4]:
plotdata = []

for dump in logdumps:
    print(f"Include dump {dump} ...")
    # Count entries to get some progress feedback
    entries = 0
    contexts = {}
    otherUserAgentContextList = []

    for entry in log_entry_iterator(filepath + dump):
        entries = entries + 1
        if (entries % 10000 == 0):
            print(f"(1/2)  processed {entries} entries")

        logentry = le.cast_log_type(entry)
        if isinstance(logentry, le.RequestLog):
            userAgent = logentry.event['request']['headers']['user-agent']
            if not (str(userAgent).startswith("node-fetch") or str(userAgent).startswith("Artillery")):
                print(f"Other user agent, will filter: {userAgent}")
                otherUserAgentContextList.append(logentry.context_id)

    entries = 0
    for entry in log_entry_iterator(filepath + dump):
        entries = entries + 1
        if (entries % 10000 == 0):
            print(f"(2/2) processed {entries} entries")

        logentry = le.cast_log_type(entry)
        if isinstance(logentry, le.PerfLog):
            if logentry.type == "measure":
                if logentry.function == "frontend":
                    if not logentry.perf_type[1] == "rpcOut":

                        id = logentry.context_id

                        # Double-Check Context
                        context_ok = True

                        if id in otherUserAgentContextList:
                            context_ok = False
                            print(f"context filter applied to context {id}")

                        if context_ok:
                            # Create context if there isn't one
                            if not (id in contexts):
                                contexts[id] = []

                            # Check if there is already an entry for this context
                            if len(contexts[id]) > 0 :
                                context_ok = False
                                for checkEntry in contexts[id]:
                                    if str(checkEntry) != str(entry):
                                        # No duplicate -> problem
                                        print(f"ERROR: {id}: {len(contexts[id])}")
                                        tmp = le.cast_log_type(checkEntry)
                                        print(f"entry: {tmp.perf_type}#{tmp.type}#{tmp.perf_type_data}#{tmp.function}")
                                        print(f"{tmp}")
                                        print(f"NEW entry:")
                                        print(f"entry: {logentry.perf_type}#{logentry.type}#{logentry.perf_type_data}#{logentry.function}")
                                        print(f"{logentry}")

                            #Parse and add data entry if context check succeeded
                            if context_ok :
                                contexts[id].append(entry)

                                function = logentry.perf_type_data
                                duration = logentry.perf["duration"]
                                plattform = logentry.platform
                                #print(f"{id}#{function}#{duration}#{plattform}")
                                row = {}
                                row["id"] = id
                                row["function"] = function
                                row["duration"] = duration
                                row["plattform"] = plattform
                                plotdata.append(row)

Include dump dump_webservice_aws_small.json ...
Iterate over ../dumps/dump_webservice_aws_small.json ...
(1/2)  processed 10000 entries
(1/2)  processed 20000 entries
(1/2)  processed 30000 entries
(1/2)  processed 40000 entries
(1/2)  processed 50000 entries
(1/2)  processed 60000 entries
Iterate over ../dumps/dump_webservice_aws_small.json ...
(2/2) processed 10000 entries
(2/2) processed 20000 entries
(2/2) processed 30000 entries
(2/2) processed 40000 entries
(2/2) processed 50000 entries
(2/2) processed 60000 entries


In [5]:
df_calls = pd.DataFrame(plotdata)
df_calls.set_index("id", drop=True, inplace=True)

df_calls.head()

,function,duration,plattform
id,,,
0zi9jvea,/,1775.917374,aws
0hfj5zsf,/setUser,18.548749,aws
l4d0u7ru,/,191.556674,aws
pee35svz,/product/:productId,1522.335138,aws
ol726tlp,/product/:productId,259.315252,aws


In [6]:
df_calls.to_pickle(oufile)